In [2]:
json_raw="""{
    "test1":  true  ,
   "test2": [true,false],
    "test3": { "name":1 }
}"""

In [3]:
def run(f):
    run=True
    pos=-1
    mem=""
    while run:
        pos+=1
        c = f.read(1)
        if not c:
            run=False
            break 
        elif c in ['[',']','{','}',",",":"]:
            if len(mem.strip()):
                yield("v",pos-len(mem),mem)
            mem=""
            yield (c,pos) #open
        elif c in ['"',"'"]:
            start_c=c
            pos_start=pos
            mem=""
            while True:
               c_prev = c
               c = f.read(1)
               pos+=1
               if not c:
                   run=False
                   break
               elif c==start_c and c_prev!="\\":
                   break
               mem+=c
            yield ('S', pos_start, mem)
            mem=""
        else:
            mem+=c


In [4]:
def print_markers(f):
    ret = []
    ids=[]
    for c in run(f):
        #print('#{:>02}'.format(len(ret)),c)
        ret.append(c)
        ids.append(c[1])

    for i,c in enumerate(json_raw):  print(c if c!="\n" else ' ',end='')
    print('')
    for i,c in enumerate(json_raw):  print(u"\u2191" if i in ids else ' ',end='')
    print('')

def print_list(f):
    i=-1
    for c in run(f):
        i+=1
        print('#{:>02} {}'.format(i,c))
        

from io import StringIO ## for Python 3
f = StringIO(json_raw)
print_markers(f)
f.seek(0)
print_list(f)

{     "test1":  true  ,    "test2": [true,false],     "test3": { "name":1 } }
↑     ↑      ↑↑       ↑    ↑      ↑ ↑↑   ↑↑    ↑↑     ↑      ↑ ↑ ↑     ↑↑ ↑ ↑
#00 ('{', 0)
#01 ('S', 6, 'test1')
#02 (':', 13)
#03 ('v', 14, '  true  ')
#04 (',', 22)
#05 ('S', 27, 'test2')
#06 (':', 34)
#07 ('[', 36)
#08 ('v', 37, 'true')
#09 (',', 41)
#10 ('v', 42, 'false')
#11 (']', 47)
#12 (',', 48)
#13 ('S', 54, 'test3')
#14 (':', 61)
#15 ('{', 63)
#16 ('S', 65, 'name')
#17 (':', 71)
#18 ('v', 72, '1 ')
#19 ('}', 74)
#20 ('}', 76)


In [5]:
from io import StringIO ## for Python 3


f = StringIO(json_raw)

out=[]

depth=0
id=[0]
for i in run(f):
    
    if i[0] in ['{','[']:
      depth+=1
      id.append(0)

    print('-'*(depth), i)
    out.append( (depth, i,id[-1]) )
    if i[0]==',':
        id[-1]+=1

    if i[0] in ['}',']']:
        depth-=1
        id.pop()

- ('{', 0)
- ('S', 6, 'test1')
- (':', 13)
- ('v', 14, '  true  ')
- (',', 22)
- ('S', 27, 'test2')
- (':', 34)
-- ('[', 36)
-- ('v', 37, 'true')
-- (',', 41)
-- ('v', 42, 'false')
-- (']', 47)
- (',', 48)
- ('S', 54, 'test3')
- (':', 61)
-- ('{', 63)
-- ('S', 65, 'name')
-- (':', 71)
-- ('v', 72, '1 ')
-- ('}', 74)
- ('}', 76)


In [18]:
f = StringIO(json_raw)

json_raw="""{
    "test1":  true  ,
    "arr":[1,2],
    "test2": { "name":1 }
}"""

a = [i for i in run(f)]
#a=[('S',0,'root'),(':',0)]+a

print( ' '.join([i[0] for i in a]) )
print('@@@@@@@@@@@@@@@@@@')
out={}

i=0

def eat_array():
    ret=[]
    global i
    while i<len(a):
        if a[i][0]==']':
            i+=1
            return ret
        elif a[i][0]==',':
            i+=1
        elif a[i][0]=='v':
            ret.append(a[i][2])
            i+=1
        else:
            raise Exception('no!',a[i:])

def eat():
    ret={}
    global i
    while i<len(a):
        if a[i][0]=='S':
            key=a[i][2]
            c,v = a[i+1],a[i+2]
            if c[0]==':' and v[0]=='{':
              i+=3
              ret[key]=eat()
            elif c[0]==':' and v[0]=='v':
              ret[key]=v[2]
              i+=3
            elif c[0]==':' and v[0]=='[':
              i+=3
              ret[key]=eat_array()
            else: raise Exception('a')
        elif a[i][0]==',': 
            i+=1
        elif a[i][0]=='{':
            i+=1
        elif a[i][0]=='}': 
            return ret
        elif a[i][0]==']': 
            return ret
        else:
            raise Exception('no',a[i:])
    return ret

r=eat()
import pprint
pprint.pprint(r)


{ S : v , S : [ v , v ] , S : { S : v } }
@@@@@@@@@@@@@@@@@@
{'arr': ['1', '2'], 'test1': '  true  ', 'test2': {'name': '1 '}}


In [55]:
from collections import deque

class TokenList:
    def __init__(self, l):
        self.l = l.copy()
        self.pos =0

    def pop(self):
        self.pos+=1
        return self.l[self.pos-1]

    def next(self):
        return self.l[self.pos] if self.pos<len(self.l) else None

    def next_is(self, v):
        return self.next()[0]== v if self.pos<len(self.l) else False
        
def eat_value(l: TokenList):
    assert l.next_is('v')
    ret = l.pop()
    return ret[2]
    
def eat_object(l:TokenList):
    ret={}
    key=None
    assert l.next_is('{'), 'not object'
    l.pop()
    while l.next():
        if l.next_is('S'):
            k,v=eat_string(l)
            ret[k]=v
        elif l.next_is(','):
            l.pop()
        elif l.next_is('}'):
            return ret
        else:
            raise Exception('array error',l.next())

def eat_array(l:TokenList):
    ret=[]
    assert l.next_is('['), 'not array'
    l.pop()
    while l.next():
        val=None
        if l.next_is('v'):
            val=eat_value(l)
        elif l.next_is('['):
            val=eat_array(l)
        elif l.next_is('{'):
            val=eat_object(l)
        elif l.next_is(','):
            l.pop()
            continue #skip iteration
        elif l.next_is(']'):
            l.pop()
            return ret
        else:
            raise Exception('array error',l.next())
        ret.append(val)

def eat_string(l: TokenList):
    assert l.next_is('S'), 'string is not string'
    key=l.pop()[2]
    assert l.next_is(':'), 'string is not string'
    l.pop()
    if l.next_is('v'):
        val=eat_value(l)
    elif l.next_is('['):
        val=eat_array(l)
    elif l.next_is('{'):
        val=eat_object(l)
    else:
        raise Exception('string error',l.next())
    return (key,val)

#############################

json_raw="""{
    "test1":  true  ,
    "arr":[1,2],
    "test2": { "name":1 }
}"""
f = StringIO(json_raw)

a = [i for i in run(f)]

l=TokenList(a)

import pprint
ret=eat_object(l)
pprint.pprint(ret)


{'arr': ['1', '2'], 'test1': '  true  ', 'test2': {'name': '1 '}}
